1. Data Augmentation de imágenes con Python

In [3]:
pip install imgaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Cargar imagen original en el notebook




In [4]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
%matplotlib inline
ia.seed(1)

image = imageio.imread("zapatilla.jpg")
ia.imshow(image)

FileNotFoundError: ignored

Rotar

In [ ]:
rotate = iaa.Affine(rotate=-45)
image_aug = rotate(image=image)

ia.imshow(image_aug)

In [ ]:
flip = iaa.Flipud(1.0)
image_aug2 = flip(image=image_aug)
ia.imshow(image_aug2)

Diversas transformaciones: rotar, flip vertical y horizontal y recorte:

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(rotate=(-45, 45)),
    iaa.Flipud(0.5),
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.2))
])
images_aug = [seq(image=image) for _ in range(12)]
ia.imshow(ia.draw_grid(images_aug, cols=4, rows=3))

2. Data Augmentation en Texto

In [ ]:
pip install textattack
pip install tensorflow_text

Embedding Augmentation

In [ ]:
from textattack.augmentation import EmbeddingAugmenter
augmenter = EmbeddingAugmenter()

s1 = 'I have a great time at the restaurant.'
augmenter.augment(s1)
s2 = 'Very good service'
augmenter.augment(s2)
s3 = 'They had really tasty food.'
augmenter.augment(s3)

# Salida
['I have a gorgeous time at the restaurant.']
['Very well service']
['They had really tasty nourishment.']

3. Oversampling en Python (Data Augmentation en Tabular)

In [ ]:
pip install imbalanced-learn
pip install pandas seaborn sklearn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
df = pd.DataFrame({"Ciudad":["Chicago","Chicago","Chicago","Chicago","Chicago","Chicago",
              "S.Antonio","S.Antonio","S.Antonio","S.Antonio","S.Antonio","S.Antonio","S.Antonio","S.Antonio","S.Antonio"],
                   "Ambientes":[3,3,2,1,1,1,
                                1,1,2,2,4,4,5,5,5],
                   "Precio":[20000,27000,15500,10000,12000,9000,
                          50000,53000,68000,65000,120000,110000,130000,125000,120000]})
df[df.Ciudad == 'Chicago']

Convertimos los nombres de las ciudades a valor numérico y lo llamaremos “label”

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
encoder = le.fit(df.Ciudad)
CiudadEncoded = encoder.transform(df.Ciudad)
df["label"] = CiudadEncoded
df

Comprobamos el desbalance de 6 contra 9 muestras

In [ ]:
sns.countplot(df['label'])

Graficamos las muestras de Chicago, el precio en relacion a la cantidad de Ambientes

In [ ]:
muestras_chicago = df[df.Ciudad=='Chicago']
sns.scatterplot(data = muestras_chicago, x ='Ambientes', y = 'Precio', hue = 'label')

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="all", random_state = 1)
X, y = smote.fit_resample(df[['Ambientes', 'Precio']], df['label'])

X_res, y_res = smote.fit_resample(X, y)

df_oversampler = pd.DataFrame(X_res, columns = ['Ambientes', 'Precio'])
df_oversampler['label'] = y_res
df_oversampler

Vemos que las tres últimas filas corresponden al label cero que es Chicago

In [ ]:
sns.countplot(df_oversampler['label'])

Veamos dónde se agregaron esas 3 nuevas muestras

In [ ]:
muestras_chicago = df_oversampler[df_oversampler.label == 0]
sns.scatterplot(data = muestras_chicago, x ='Ambientes', y = 'Precio', hue = 'label')

plt.scatter(df_oversampler.Ambientes[15:], df_oversampler.Precio[15:], c='red', s=20)